In [165]:
import pandas as pd
import numpy as np

# Reads a path to .tsv file, returns the bitmatrix, 
# indexes pointing to fasta files (sort of redundant)
# and clique member names in order relative to bitmatrix
def read_bitmatrix(path):
    tsv_file = pd.read_csv(path, sep='\t').sort_values('query_name')
    bitmat = tsv_file.values[:,1:]
    index = tsv_file.values[:,0]
    members = list(tsv_file.columns[1:])
    return(bitmat,index,members)

# From a bitmatrix, get a submatrix only consisting of unique
# vector forms founds in the original matrix
# Returns a reduced matrix consisting of every variation of 
# row vector, i.e. every way a node in CCDBG can be colored 
# differently, as well as a reference list which maps the 
# row index of a vector in the reduced matrix to the indices of
# that vector form's corresponding fasta sequences.
# Also returns members, a list of member names in order
# of appearance in bitmatrix columns.
def reduce_bit_matrix(path):
    bitmat, index, members = read_bitmatrix(path)
    uniques = set()
    mapping = dict()
    for i in range(bitmat.shape[0]):
        if (bitmat[i] == 0).any():
            form = tuple(bitmat[i])
            uniques.add(form)
            if form in mapping:
                mapping[form].append(index[i])
            else:
                mapping[form] = [index[i]]
    N = len(uniques)
    M = len(members)
    reduced = np.zeros((N,M))
    degrees = dict()
    connected_to = dict()
    ref_to_fasta = list()
    uniques = sorted(uniques, key=sum)
    for i,form in enumerate(uniques):
            reduced[i] = np.array(form)
            ref_to_fasta.append(mapping[form])
            deg = sum(form)
            if deg in degrees:
                degrees[deg].append(i)
            else:
                degrees[deg] = [i]
            for mem in np.where(reduced[i] == 1)[0]:
                if mem in connected_to:
                    connected_to[mem].append(i)
                else:
                    connected_to[mem] = [i]
    return(reduced, ref_to_fasta, members, degrees, connected_to)

def first_pass(reduced, fasta_indices, degree_log):
    M = reduced.shape[1]

    connections = np.zeros((M,M))
    chosen = dict()
    for seq in degree_log[1]:
        respective_member = int(np.where(reduced[seq] == 1)[0])
        for i, fasta in enumerate(fasta_indices[seq]):
            if respective_member in chosen:
                chosen[respective_member].append(fasta)
            else:
                chosen[respective_member] = [fasta]
            connections[respective_member] += reduced[seq]
            i+=1
            if i == 10:
                break
    return(chosen, connections)

def second_pass(reduced, fasta_indices, degree_log, chosen, connections, connected_to):
    deg = 2
    while (sum(connections) < 10).any():
        focus_mem = np.argmin(sum(connections))
        options = list(set(connected_to[focus_mem]).intersection(set(degree_log[deg])))
        chosen_index = np.argmin(np.matmul(reduced[options],connections[focus_mem]))
        chosen_vec = options[chosen_index]
        chosen_fasta = fasta_indices[chosen_vec][0]
        chosen.append(chosen_fasta)
        fasta_indices[chosen_vec].remove(chosen_fasta)
        respective_members = np.where(reduced[chosen_vec] == 1)[0]
        for respective_member in respective_members:
            if respective_member in chosen:
                chosen[respective_member].append(chosen_fasta)
            else:
                chosen[respective_member] = [chosen_fasta]
            connections[respective_member] += reduced[chosen_vec]
        deg+=1
    return(chosen, connections)
    
    

def parse_fasta(fasta_path):
        desc = []
        seqs = []

        s = []
        for line in open(fasta_path, 'r'):
            if line[0] == '>':
                if s:
                    seqs.append(''.join(s))
                s = []
            else:
                s.append(line.strip())

        seqs.append(''.join(s))
        
        return {'file': fasta_path, 'seqs': seqs}

def print_to_file(chosen, fas, mem, fasta_path):
    fastas = parse_fasta(fasta_path)['seqs']
    seq_file_path = fasta_path.split('.')[0]+'.seqs'
    with open(seq_file_path, 'w') as f:
        for mem_ind in chosen:
            f.write(mem[mem_ind])
            f.write(':\n')
            for fas_ind in chosen[mem_ind]:
                fasta_seq = fastas[fas_ind][:31]
                f.write(fasta_seq)
                f.write('\n')
            f.write('\n')

def choose_sequences(path):
    fasta_path = path.split('.')[0]+'.clique.fasta'
    
    red, fas, mem, deg, connected_to = reduce_bit_matrix(path)
    chosen, connections = first_pass(red, fas, deg)
    
    if (sum(connections) == 10).all():
        print_to_file(chosen, fas, mem, fasta_path)
    else:
        chosen, connections = second_pass(red, fas, deg, chosen, connections, connected_to)
        print_to_file(chosen, fas, mem, fasta_path)
    print(path,'done...')
    return(red, chosen, connections, deg, mem)

In [166]:
reduced, chosen, conn, degs, mem = choose_sequences('82040.clique.tsv')
reduced.shape

82040.clique.tsv done...


(33, 6)

In [133]:
deg_1_span = np.where(np.sum(reduced, axis=1) == 2)[0][0]
deg_1_span

6

In [156]:
for mems in chosen:
    print(mem[mems].split('/')[-1].split('_genomic')[0])

GCF_001708775.1_ASM170877v1
GCF_003608045.1_ASM360804v1
GCF_001827025.1_ASM182702v1
GCF_003607805.1_ASM360780v1
GCF_012063895.1_ASM1206389v1
GCF_001828155.1_ASM182815v1


Eflaust pínku bull en fyrir n sæta tvíundatölu k, þá kemur talan k næst aftur fyrir í tölunni k+2^n.

Þetta þýðir að fyrir töluna 10 = 2 er n = 2 og þá kemur 10 næst aftur fyrir í tölunni 110 = 6 = 2+4 = 2+2^2.
Fyrir töluna 11 = 3 er n = 2 og þá kemur 11 næst aftur fyrir í tölunni 111 = 7 = 3+4 = 2+2^2.
Fyrir töluna 100 = 4 er n = 3 og þá kemur 100 næst aftur fyrir í tölunni 1100 = 12 = 4+8 = 4+2^3.

Þetta þýðir eftirfarandi: Fyrir gefið n x d fylki A, þar sem n er fjöldi erfðamengja og d er fjöldi einkennisraða, getur verið að við viljum skoða einstaklinginn sem tilheyrir dálki i í fylkinu. Skilgreinum nú b = n-(i+1). Allir bitavigrar sem þessi einstaklingur kemur fram í eru þá jafngildir þeim tugatölum, T, sem uppfylla 

T = 2^b + n \* 2^(b+1).

Hvað einkennir góðar niðurstöður hér?

Byrjum með n x d fylki A.
Smækkum það niður í n x m möguleikafylkið D.

Viljum finna n x V undirfylkið K með V völdum dálkum, þar sem

    markmið: raðarsumma K er nálægt eða jöfn 10
        þetta er jafngilt því að hornalínan í tengslafylkinu sé öll með >= 10
    skorða: 